In [183]:
#台北市URL
import requests
import json
from bs4 import BeautifulSoup as bs
f = open('591_2.txt','w')
page_html = "http://sale.591.com.tw/index.php?module=search&action=rslist&is_new_list=1&type=2&searchtype=1&region=1&orderType=desc&listview=img&shType=list&kind=9&firstRow={0}&totalRows=11678"
res = requests.get(page_html)
count = json.loads(res.text)
cases = ''.join(count['count'].split(','))
pages = int(ceil(float(cases)/20))
for i in range(0,pages+1):
    i = 20 * i
    res = requests.get(page_html.format(i))
    data = json.loads(res.text)
    soup = bs(data['main'])
    for ele in soup.select('.shList'):
        f.write("http://sale.591.com.tw/"+ele.select('a')[0]['href']+'\n')
f.close()

In [170]:
#計算URL
line_num = 0
f = open("591_2_all.txt","r")
for line in f.readlines():
    line_num = line_num + 1
print line_num
f.close()

11685


In [177]:
#將重複的物件讀出
case_dic = {}
f = open('591_2_all.txt','r')
re = open('re_2.txt','w')
for line in f.readlines():
    rec = line.strip()
    caseno = rec.split('/')[3]
    if caseno not in case_dic:
        case_dic[caseno] = 1
    else:
        re.write(caseno+'\n')
re.close()
f.close()

In [473]:
#存入各物件html+經緯度
dir = '591'
f = open('591_2.txt','r')
for line in f:
    file_name = line.strip().split('/')[3].split('.')[0].split('-')[2]
    input_file = open(dir+'/'+file_name+'.txt','w')
    
    res = requests.get(line.strip())
    response = res.text.encode('utf8')
    soup = bs(response)
    map_page = 'http://sale.591.com.tw/'+soup.select('#mapRound iframe')[0]['src']
    map_pages = requests.get(map_page)
    soup = bs(map_pages.text)
    latlngs = soup.select('.propMapBarMap iframe')[0]['src'].split('&q=')[1].split('&z=')[0].split(',')
    
    input_file.write(latlngs[0]+','+latlngs[1]+'\001'+'\n'+response)
input_file.close()
f.close()
#http://sale.591.com.tw/map-houseRound.html?type=2&post_id=3580915&detail=detail

In [182]:
#練習selenium 抓HTML
from selenium.webdriver.common.by import By
import selenium.webdriver as webdriver
import time
from bs4 import BeautifulSoup as bs 

#print dir(webdriver)
driver = webdriver.Chrome("/Users/Derek/Downloads/chromedriver")
f = open('591_2.txt','r')
for line in f:
    file_name = line.strip().split('/')[3].split('.')[0]
    input_file = open(file_name+'.txt','w')
    driver.get(line.strip())
    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #driver.execute_script("scroll(0.250);")
    response = driver.page_source.encode('utf8')
    input_file.write(response)
    time.sleep(2)
driver.close()
input_file.close()
f.close()

In [673]:
‪#在資料夾中逐一讀取TXT檔案裡的HTML‬
import glob
import os

path = '/Users/Derek/Project@iii/591/'
for file_name in glob.glob(os.path.join(path, "*.txt")):
    f = open(file_name, "r")
    soup = bs(f.read())

In [ ]:
# road:所在路段
# latitude:緯度
# longitude:經度
# price:總價(萬)
# price_per:每坪單價，若是0則需計算
# Housing_pattern_bedroom:房(室)
# Housing_pattern_hall:廳
# Housing_pattern_toilet:衛
# age:屋齡
# floor:樓層
# building_height:樓高
# building_type:類型
# elevator:電梯數目
# orientation:座向/方位
# area_total:總面積(總坪數)/建物登記坪數
# main_building:主建物坪數
# establishes_with_whole:公設坪數
# parking_space:車位種類/有無
# community:社區
# building_structure:建築結構

In [476]:
#屬性擷取(欄位不固定，失敗！！)
import requests
from bs4 import BeautifulSoup
import re
import glob
import os

#res = requests.get('http://sale.591.com.tw/sale-detail-2444655.html')
#soup = bs(res.text)
ff = open('591_detail.txt','w')
dir_path = '/Users/Derek/Project@iii/591/'
for file_name in glob.glob(os.path.join(dir_path, "*.txt")):
    f = open(file_name, "r")
    text = open(file_name, "r").read()
    soup = bs(f.read())
#Primary Key    
    pk = re.search('S([\d]*)',str(soup.select('#propNav')[0].text.encode('utf8'))).group(1)
#緯經度
    latitude = text.split('\001')[0].split(',')[0]
    longitude = text.split('\001')[0].split(',')[1]
# 總價
    price = re.search('([\d\,]*) 萬元',str(''.join(soup.select('#attr li')[0].select('span')[1].text.encode('utf8').split(',')))).group(1)
#price = ''.join(soup.select('#attr li')[0].select('span')[1].text.split(','))
#每坪單價
    price_per = re.search('([\d\.]*)萬元/坪',str(soup.select('#attr li')[1].select('span')[1].text.encode('utf8'))).group(1)
#格局
    #Housing_pattern_bedroom = re.search('([\d]*)房',str(soup.select('#attr li')[2].select('span')[1].text.encode('utf8'))).group(1)
    #Housing_pattern_hall = re.search('([\d]*)廳',str(soup.select('#attr li')[2].select('span')[1].text.encode('utf8'))).group(1)
    #Housing_pattern_toilet = re.search('([\d]*)衛',str(soup.select('#attr li')[2].select('span')[1].text.encode('utf8'))).group(1)
#總坪數
    area_total = re.search('([\d\.]*)坪',str(soup.select('#attr li')[3].select('span')[1].text.encode('utf8'))).group(1)
#樓層/樓高
    floor = re.search('([\d]*)F',str(soup.select('#attr li')[4].select('span')[1].text.encode('utf8'))).group(1)
    building_height = re.search('/([\d]*)F',str(soup.select('#attr li')[4].select('span')[1].text.encode('utf8'))).group(1)
#屋齡
    age = re.search('([\d]*)年',str(soup.select('#attr li')[5].select('span')[1].text.encode('utf8'))).group(1)
#類型
    building_type = soup.select('#attr li')[7].select('span')[1].text.split('/')[0].encode('utf8')
#車位 (車位種類？)
#parking_space = soup.select('#attr li')[8].select('span')[1].text
    #parking_space = '1' if re.search('坪',str(soup.select('#attr li')[8].select('span')[1].text.encode('utf8'))) else '0'
#社區
    #community = soup.select('#attr li')[9].select('span')[1].text
#所在路段
    #road = soup.select('#attr li')[9].select('span')[1].text
    road = re.search('([\S]*)',str(soup.select('#attr')[0].select('.addr')[0].text.encode('utf8'))).group(1)
#方位
    #orientation = unicode(re.search('朝向：([\S]*)',str(soup.select('#sale-other-info div')[1].text.encode('utf8'))).group(1),'utf8')
#主建物坪數
    #main_building = re.search('坪數說明：主建物([\d\.]*)坪、附屬建物([\d\.]*)坪、共用部分([\d\.]*)坪、公設比([\d\D]*)',str(soup.select('#sale-other-info div')[4].text.encode('utf8'))).group(1)
#公設比
    #establishes_with_whole = re.search('坪數說明：主建物([\d\.]*)坪、附屬建物([\d\.]*)坪、共用部分([\d\.]*)坪、公設比([\d\D]*)',str(soup.select('#sale-other-info div')[4].text.encode('utf8'))).group(4)

    ff.write(pk+','+latitude+','+longitude+','+road+','+price+','+price_per \
            +','+age+','+floor+','+building_height+','+building_type+','+area_total)
    ff.write('\n')
    #print latitude+','+longitude
    f.close()
ff.close()
#print road+','+price+','+price_per+','+Housing_pattern_bedroom+','+Housing_pattern_hall+','+Housing_pattern_toilet \
#    +','+age+','+floor+','+building_height+','+building_type+','+orientation+','+area_total+','+main_building \
#    +','+establishes_with_whole+','+parking_space+','+community


In [37]:
#屬性擷取(完全使用正規表示法加上判斷式，車位判定未完成)
import requests
from bs4 import BeautifulSoup as bs
import re
import glob
import os

    # ==================================
pk = ''
road = ''
latitude = ''
longitude = ''
price = ''
price_per =''
Housing_pattern_bedroom = ''
Housing_pattern_hall = ''
Housing_pattern_toilet = ''
age = ''
floor = ''
building_height = ''
building_type = ''
elevator = ''
orientation = ''
area_total = ''
main_building = ''
establishes_with_whole = ''
parking_space = ''
community = ''
building_structure = ''

try:
    ff = open('591_detail.txt','w')
    dir_path = '/Users/Derek/Project@iii/591/'
    for file_name in glob.glob(os.path.join(dir_path, "*.txt")):
        f = open(file_name, "r")
        text = open(file_name, "r").read()
        soup = bs(f.read())
#Primary Key    
        pk = re.search('S([\d]*)',str(soup.select('#propNav')[0].text.encode('utf8'))).group(1)
#緯經度
        latitude = text.split('\001')[0].split(',')[0]
        longitude = text.split('\001')[0].split(',')[1]
    # =================主要屬性值=================
        string = ''
        for ele in soup.select('#attr')[0].select('li'):
            string = string + ' ' + ' '.join(ele.text.split())
# 總價
        price = re.search('售金：([\d\,]*) 萬元',str(''.join(string.encode('utf8').split(',')))).group(1)
#每坪單價    
        if re.search('單價：([\d\.]*)萬',str(string.encode('utf8'))):
            price_per = re.search('單價：([\d\.]*)萬',str(string.encode('utf8'))).group(1)  
        else: price_per = ''
#格局
        if re.search('格局：([\d]*)房',str(string.encode('utf8'))):
            Housing_pattern_bedroom = re.search('格局：([\d]*)房',str(string.encode('utf8'))).group(1)
        else:Housing_pattern_bedroom = ''
        
        if re.search('([\d]*)廳',str(string.encode('utf8'))):
            Housing_pattern_hall = re.search('([\d]*)廳',str(string.encode('utf8'))).group(1)
        else:Housing_pattern_hall = ''
        
        if re.search('([\d]*)衛',str(string.encode('utf8'))):
            Housing_pattern_toilet = re.search('([\d]*)衛',str(string.encode('utf8'))).group(1)
        else:Housing_pattern_toilet = ''
#總坪數
        if re.search('權狀坪數：([\d\.]*)坪',str(string.encode('utf8'))):
            area_total = re.search('權狀坪數：([\d\.]*)坪',str(string.encode('utf8'))).group(1)
        else:area_total = ''
#樓層/樓高
        if re.search('樓層：([\d]*)F',str(string.encode('utf8'))):
            floor = re.search('樓層：([\d]*)F',str(string.encode('utf8'))).group(1)
        elif re.search('樓層：([\S]*)/',str(string.encode('utf8'))):
            floor = re.search('樓層：([\S]*)/',str(string.encode('utf8'))).group(1)
        else: floor = ''
        
        if re.search('/([\d]*)F',str(string.encode('utf8'))):
            building_height = re.search('/([\d]*)F',str(string.encode('utf8'))).group(1)
        else: building_height = ''
#屋齡
        if re.search('屋齡：([\d]*)年',str(string.encode('utf8'))):
            age = re.search('屋齡：([\d]*)年',str(string.encode('utf8'))).group(1)
        else: age = ''
#類型
        if re.search('型態/類型：([\S]*)/',str(string.encode('utf8'))):
            building_type = re.search('型態/類型：([\S]*)/',str(string.encode('utf8'))).group(1)
        else: building_type =''
#車位
        if re.search('車位：([\S]*) ',str(string.encode('utf8'))):
            parking_space = re.search('車位：([\S]*) ',str(string.encode('utf8'))).group(1)
        else: parking_space = '-1'
#社區
        if re.search('社區：([\S]*) ',str(string.encode('utf8'))):
            community = re.search('社區：([\S]*) ',str(string.encode('utf8'))).group(1)
        else: community = ' '
#所在路段
        if re.search('地址：([\S]*)',str(string.encode('utf8'))):
            road = re.search('地址：([\S]*)',str(string.encode('utf8'))).group(1)
        else: road = ''
    
    # =================其他屬性值=================
        other = ''
        for eles in soup.select('#sale-other-info')[0].select('div'):
            other = other + ' ' + ' '.join(eles.text.split())
#方位
        if re.search('朝向：([\S]*) ',str(other.encode('utf8'))): 
            orientation = unicode(re.search('朝向：([\S]*) ',str(other.encode('utf8'))).group(1),'utf8')
        else: orientation = ' '
#主建物坪數
        if re.search('主建物([\d\.]*)坪',str(other.encode('utf8'))): 
            main_building = re.search('主建物([\d\.]*)坪',str(other.encode('utf8'))).group(1)
        else: main_building = ' '
#公設比
        if re.search('公設比([\d\S]*) ',str(other.encode('utf8'))):
            establishes_with_whole = re.search('公設比([\d\S]*) ',str(other.encode('utf8'))).group(1)
        else: establishes_with_whole = ' '

        ff.write(pk+','+road+','+latitude+','+longitude+','+price+','+price_per+','+Housing_pattern_bedroom \
            +','+Housing_pattern_hall+','+Housing_pattern_toilet+','+age+','+floor+','+building_height \
            +','+building_type+','+area_total+','+main_building+','+establishes_with_whole+','+parking_space \
            +','+community)
        ff.write('\n')
    f.close()
    ff.close()

except:
    print pk,'err'
        
    